# 데이터 전처리 및 피처 엔지니어링

In [0]:
root= '/FileStore/tables/bin'
retail = root+'/retail-data/by-day/*.csv'
simple_ml_int = root+'/simple-ml-integers'
simple_ml = root+'/simple-ml'
simple_ml_scale = root+'/simple-ml-scaling'

In [0]:
sales = spark.read.format('csv')\
.option('header', 'true')\
.option('inferSchema', 'true')\
.load(retail)\
.coalesce(5)\
.where("Description is not null")

In [0]:
fakeIntDF = spark.read.parquet(simple_ml_int)
simpleDF = spark.read.format('json').load(simple_ml)
scaleDF = spark.read.parquet(simple_ml_scale)

In [0]:
display(fakeIntDF)

int1,int2,int3
1,2,3
4,5,6
7,8,9


In [0]:
display(simpleDF.head(10))

color,lab,value1,value2
green,good,1,14.386294994851129
blue,bad,8,14.386294994851129
blue,bad,12,14.386294994851129
green,good,15,38.97187133755819
green,good,12,14.386294994851129
green,bad,16,14.386294994851129
red,good,35,14.386294994851129
red,bad,1,38.97187133755819
red,bad,2,14.386294994851129
red,bad,16,14.386294994851129


In [0]:
display(scaleDF)

id,features
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))"
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))"
1,"List(1, 3, List(), List(3.0, 10.1, 3.0))"


In [0]:
sales.cache()
display(sales.head(10))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
579256,20973,12 PENCIL SMALL TUBE WOODLAND,24,2011-11-29 08:07:00,0.65,13349.0,United Kingdom
579256,21891,TRADITIONAL WOODEN SKIPPING ROPE,12,2011-11-29 08:07:00,1.45,13349.0,United Kingdom
579256,22398,MAGNETS PACK OF 4 SWALLOWS,12,2011-11-29 08:07:00,0.39,13349.0,United Kingdom
579256,21746,SMALL RED RETROSPOT WINDMILL,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,21747,SMALL SKULL WINDMILL,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,22585,PACK OF 6 BIRDY GIFT TAGS,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,84228,HEN HOUSE WITH CHICK STANDING,12,2011-11-29 08:07:00,0.42,13349.0,United Kingdom
579256,21894,POTTING SHED SEED ENVELOPES,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,23433,HANGING QUILTED PATCHWORK APPLES,12,2011-11-29 08:07:00,0.83,13349.0,United Kingdom
579256,23476,WOODLAND LARGE RED FELT HEART,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom


## 사용 목적에 따라 모델 서식 지정하기
- 데이터를 전처리하기 위해서는 우선 <strong>최종 목표</strong>를 검토해야함
- MLlib의 각 고급 분석 작업을 위한 입력 데이터 구조 관련 요구사항
  - 대부분의 <strong>분류 및 회귀 알고리즘</strong>의 경우 데이터를 <strong>Double</strong>타입의 컬럼으로 가져와서 레이블을 표시하고 <strong>Vector</strong>타입(밀도가 높거나 희소한)의 컬럼을 사용하여 특징을 나타내야함
  - <strong>추천 알고리즘</strong>의 경우 데이터를 사용자 컬럼, 영화 또는 서적 등을 나타내는 아이템 컬럼 그리고 사용자 평점 등을 나타내는 등급 컬럼으로 표현해야함
  - <strong>비지도 학습 알고리즘</strong>의 경우 입력 데이터로 사용할 특징을 <strong>Vector</strong>타입의 컬럼으로 표현해야함
  - <strong>그래프 분석</strong>의 경우 정점과 에지가 각각 <strong>DataFrame</strong>으로 구성되어야함

## 변환자
- <strong>다양한 방식으로 원시 데이터를 변환</strong>시키는 함수
  - 주로 데이터 전처리 또는 특징 생성을 위해 사용
  - 모든 변환자는 입력과 출력의 컬럼 이름을 나타내는 'inputCol'과 'outputCol'을 지정해야함
    - set 메서드로
- 새로운 상호작용 변수를 생성(두 개의 다른 변수로)하거나 컬럼을 정규화하거나 모델에 입력하기 위한 변수 타입 변경 등
- ex) Tokenizer
  - 문자열을 토큰화하고 주어진 문자로 분할함

## 전처리 추정자
- 수행하려는 변환이 입력 컬럼에 대한 데이터 또는 정보로 초기화되어야 할 때 필요
- 추정자는 <strong>특정 입력 데이터에 따라 구성되는 변환자</strong>라고 볼 수 있음
- ex) StandardScaler
  - 입력 컬럼에 적재된 모든 값의 범위를 고려하여 크기를 조정하여 평균이 0, 분산이 1이 되도록 스케일링 수행

## 고수준 변환자
- 일반적으로 <strong>오류 위험을 최소화</strong>하고, 구현 과정의 세세한 부분보다 <strong>비즈니스 문제에 집중하기 위해</strong> 가능한 한 최상위 수준의 변환자를 사용해야함
- <strong>RFormula</strong> 같은 고수준 변환자를 사용하면 하나의 변환에서 <strong>여러 가지 변환을 간결하게 지정</strong>할 수 있음
- 고수준 변환자는 '상위 수준'에서 동작하며 데이터 조작 및 변환을 하나하나 수행하지 않도록 해줌

### RFormula
- R언어에서 빌려온 변환자로서 데이터에 대한 변환을 선언적으로 간단히 지정할 수 있게 해줌
  - 문법만 이해하면 매우 쉽게 사용 가능
- 숫자컬럼은 Double타입으로 변환되지만 원-핫 인코딩은 되지 않음
- 또한 레이블 컬럼이 String인 경우 먼저 StringIndexer를 사용해서 Double타입으로 변환됨
- [기본 연산자](https://lovetoken.github.io/r/2016/12/06/formula_usage.html)

In [0]:
from pyspark.ml.feature import RFormula

#모든 변수를 사용(.)하고 value1과 color, value2와 color간의 상호작용을 지정하여 새로운 특징 생성
supervised = RFormula(formula ="lab ~ . + color:value1 + color:value2")
fitted_sup = supervised.fit(simpleDF)

In [0]:
display(fitted_sup.transform(simpleDF).head(10))

color,lab,value1,value2,features,label
green,good,1,14.386294994851129,"List(0, 10, List(1, 2, 3, 5, 8), List(1.0, 1.0, 14.386294994851129, 1.0, 14.386294994851129))",1.0
blue,bad,8,14.386294994851129,"List(0, 10, List(2, 3, 6, 9), List(8.0, 14.386294994851129, 8.0, 14.386294994851129))",0.0
blue,bad,12,14.386294994851129,"List(0, 10, List(2, 3, 6, 9), List(12.0, 14.386294994851129, 12.0, 14.386294994851129))",0.0
green,good,15,38.97187133755819,"List(0, 10, List(1, 2, 3, 5, 8), List(1.0, 15.0, 38.97187133755819, 15.0, 38.97187133755819))",1.0
green,good,12,14.386294994851129,"List(0, 10, List(1, 2, 3, 5, 8), List(1.0, 12.0, 14.386294994851129, 12.0, 14.386294994851129))",1.0
green,bad,16,14.386294994851129,"List(0, 10, List(1, 2, 3, 5, 8), List(1.0, 16.0, 14.386294994851129, 16.0, 14.386294994851129))",0.0
red,good,35,14.386294994851129,"List(0, 10, List(0, 2, 3, 4, 7), List(1.0, 35.0, 14.386294994851129, 35.0, 14.386294994851129))",1.0
red,bad,1,38.97187133755819,"List(0, 10, List(0, 2, 3, 4, 7), List(1.0, 1.0, 38.97187133755819, 1.0, 38.97187133755819))",0.0
red,bad,2,14.386294994851129,"List(0, 10, List(0, 2, 3, 4, 7), List(1.0, 2.0, 14.386294994851129, 2.0, 14.386294994851129))",0.0
red,bad,16,14.386294994851129,"List(0, 10, List(0, 2, 3, 4, 7), List(1.0, 16.0, 14.386294994851129, 16.0, 14.386294994851129))",0.0


### SQL 변환자
- SQL Transformer를 사용하면 MLlib 변환 기능을 사용할 때처럼 스파크의 방대한 SQL데이터 처리 라이브러리를 활용할 수 있음
  - 근데 테이블 이름 대신 THIS 키워드를 사용함
- 사용자는 원시 데이터로부터 현재 상태에 이르기까지의 모든 조작을 SQLTransformer를 사용하여 적용할 수 있음
- 이를 위해 사용자는 각각의 조작을 변환자로 <strong>버전화</strong>하여 식별 가능
  - 따라서 변환자만 교체하면 <strong>다양한 파이프라인을 구축하고 테스트 가능</strong>

In [0]:
from pyspark.ml.feature import SQLTransformer
basicTransformer = SQLTransformer()\
.setStatement('''
select sum(quantity), count(*), customerID
from __THIS__
group by customerID
''')

In [0]:
display(basicTransformer.transform(sales).head(10))

sum(quantity),count(1),customerID
223,20,17633.0
107,43,13533.0
123,9,13937.0
2138,23,16656.0
187,15,15145.0
13748,822,15311.0
466,39,17659.0
-3,2,13763.0
575,27,16353.0
67,13,14211.0


### 벡터 조합기
- VectorAssembler는 사용자가 생성하는 거의 모든 <strong>단일 파이프라인</strong>에서 사용하게 될 도구
  - <strong>모든 특징을 하나의 큰 벡터로 연결</strong>하여 추정자에게 전달하는 기능 제공
  - <strong>다양한 변환자를 사용하여 여러 가지 조작</strong>을 수행하고 그에 대한 <strong>모든 결과를 모아야하는 경우</strong>에 특히 유용
- 일반적으로 머신러닝 파이프라인의 마지막 단계에서 사용되고 boolean, double, vector와 같은 다양한 컬럼을 입력으로 사용

In [0]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(['int1', 'int2','int3'])
va.transform(fakeIntDF).show()

+----+----+----+------------------------------------+
int1|int2|int3|VectorAssembler_7e9fa2e28926__output|
+----+----+----+------------------------------------+
 1| 2| 3| [1.0,2.0,3.0]|
 4| 5| 6| [4.0,5.0,6.0]|
 7| 8| 9| [7.0,8.0,9.0]|
+----+----+----+------------------------------------+

## 연속형 특징 처리하기
- 연속형 특징을 처리하는 데 일반적으로 사용되는 두 개의 변환자가 있음
  1. 버켓팅
  2. 스케일링과 정규화

In [0]:
#일단 모든 변환자는 Double타입에서만 작동
contDF = spark.range(20).selectExpr("cast(id as double)")

### 버켓팅
- <strong>버켓팅 또는 구간화(binning)</strong>에 대한 가장 직접적인 접근법은 <strong>Bucketizer</strong>를 사용하는 것
- Bucketizer를 사용하면 주어진 연속형 특징을 지정한 버켓으로 분할함
  - 이때 <strong>어떤 형태로 생성</strong>되어야 하는지 <strong>Double타입 값으로 된 배열이나 리스트로 지정</strong> 가능
  - 이는 나중에 데이터셋의 특징을 단순화하거나 차후의 해석을 위해 표현을 단순화하려는 경우에 유용
  - ex) 사람의 체중을 나타내는 컬럼 -> 과체중 / 평균 / 저체중의 세 가지 버켓으로 나누어 활용하는 것이 더 간단한 접근 방법일 수 있음
- 버켓을 지정하려면 경계를 설정해야하는데 다음 세 가지 요구사항을 충족해야함
  1. 분할 배열의 최솟값은 DF의 최솟값보다 작아야함
  2. 분할 배열의 최댓값은 DF의 최댓값보다 커야함
  3. 분할 배열은 최소 세 개 이상의 값을 지정해서 두 개 이상의 버켓을 만들도록 해야함

In [0]:
from pyspark.ml.feature import Bucketizer

#근데 가능한 모든 범위를 포함하기 위해 float('inf'), float('-inf')로 지정하는 방법도 있음
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol('id')
bucketer.transform(contDF).show()

+----+-------------------------------+
 id|Bucketizer_00dcfd0fba78__output|
+----+-------------------------------+
 0.0| 0.0|
 1.0| 0.0|
 2.0| 0.0|
 3.0| 0.0|
 4.0| 0.0|
 5.0| 1.0|
 6.0| 1.0|
 7.0| 1.0|
 8.0| 1.0|
 9.0| 1.0|
10.0| 2.0|
11.0| 2.0|
12.0| 2.0|
13.0| 2.0|
14.0| 2.0|
15.0| 2.0|
16.0| 2.0|
17.0| 2.0|
18.0| 2.0|
19.0| 2.0|
+----+-------------------------------+

In [0]:
#위처럼 하드코딩된 값을 기반으로 분할하는 방법 외에도, 백분위수를 기준으로 분할하는 방법도 있음
from pyspark.ml.feature import QuantileDiscretizer
bucketer_q = QuantileDiscretizer().setNumBuckets(5).setInputCol('id').setOutputCol('result')

In [0]:
fitted_bucketer = bucketer_q.fit(contDF)
fitted_bucketer.transform(contDF).show()

+----+------+
 id|result|
+----+------+
 0.0| 0.0|
 1.0| 0.0|
 2.0| 0.0|
 3.0| 1.0|
 4.0| 1.0|
 5.0| 1.0|
 6.0| 1.0|
 7.0| 2.0|
 8.0| 2.0|
 9.0| 2.0|
10.0| 2.0|
11.0| 3.0|
12.0| 3.0|
13.0| 3.0|
14.0| 3.0|
15.0| 4.0|
16.0| 4.0|
17.0| 4.0|
18.0| 4.0|
19.0| 4.0|
+----+------+

## 범주형 특징 처리하기

## 텍스트 데이터 변환자

## 특징 조작하기

## 특징 선택

## 고급 주제